In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [45]:
data = pd.read_excel('HARGA RUMAH JAKSEL.xlsx', sheet_name='Sheet1',skiprows=1)
data.head(10)

,HARGA,LT,LB,JKT,JKM,GRS,KOTA
0,28000000000,1100,700,5,6,ADA,JAKSEL
1,19000000000,824,800,4,4,ADA,JAKSEL
2,4700000000,500,400,4,3,ADA,JAKSEL
3,4900000000,251,300,5,4,ADA,JAKSEL
4,28000000000,1340,575,4,5,ADA,JAKSEL
5,10000000000,460,300,4,4,ADA,JAKSEL
6,7600000000,278,350,4,4,ADA,JAKSEL
7,5250000000,511,300,3,2,ADA,JAKSEL
8,670000000,70,69,3,2,TIDAK ADA,JAKSEL
9,480000000,66,42,2,1,TIDAK ADA,JAKSEL


In [46]:
pd.DataFrame(data.HARGA.describe()).T

,count,mean,std,min,25%,50%,75%,max
HARGA,1001.0,1.747472e+10,2.079548e+10,430000000.0,6.750000e+09,1.350000e+10,2.000000e+10,2.500000e+11


## BIVARIAT

In [47]:
px.histogram(
    data_frame=data, x ='LT', y = 'HARGA',
    nbins=5,histfunc='avg',color_discrete_sequence=['black'],
    title='Perbandingan Harga Dengan Luas Tanah'
            )

In [54]:
px.histogram(data_frame=data,x='LB',y='HARGA',
             title='Harga Rumah Jakarta Selatan Berdasarkan Luas Bangunan',
             histfunc='avg',
             nbins=4,color_discrete_sequence=['#749F82']
             )

## UNIVARIAT

In [100]:
pd.DataFrame(data['GRS'].value_counts().reset_index())

,GRS,count
0,ADA,779
1,TIDAK ADA,222


In [108]:
px.pie(data_frame=data['GRS'].value_counts().reset_index(), 
       values = 'count',
       names = 'GRS',
       title = 'Jumlah Rumah Jaksel yang Ada Garasi'
       )

In [137]:
pd.DataFrame(data['JKT'].describe()).T

,count,mean,std,min,25%,50%,75%,max
JKT,1001.0,4.457542,2.004606,1.0,4.0,4.0,5.0,27.0


In [136]:
dataKamarTidur = pd.DataFrame(
    data['JKT'].value_counts().reset_index().rename(columns={'JKT' :'JKT','count': 'Banyak Rumah'})
    )
px.bar(
    data_frame=dataKamarTidur, x = 'JKT', y = 'Banyak Rumah',
    title = 'Jumlah Kamar Tidur Rumah Jaksel'
)

In [142]:
dataKamarMandi = pd.DataFrame(
    data['JKM'].value_counts().reset_index().rename(columns={'count':'Banyak Rumah'})
  )
px.bar(
    data_frame=dataKamarMandi, y = 'Banyak Rumah', x = 'JKM',
    title = 'Jumlah Kamar Mandi Rumah Jaksel'
)

In [143]:
data['KOTA'].value_counts()

KOTA
JAKSEL    1001
Name: count, dtype: int64

## PREPROCESSING

In [144]:
data.drop(columns='KOTA',inplace=True,axis=1)

In [146]:
data.GRS = data.GRS.map({'ADA':1,'TIDAK ADA':0})

In [147]:
data = data[(data.LT>data.LT.quantile(0.01))&(data.LT<data.LT.quantile(0.99))]
data = data[(data.LB>data.LB.quantile(0.01))&(data.LB<data.LB.quantile(0.99))]
data = data[(data.JKT>data.JKT.quantile(0.01))&(data.JKT<data.JKT.quantile(0.99))]
data = data[(data.JKM>data.JKM.quantile(0.01))&(data.JKM<data.JKM.quantile(0.99))]
data = data[(data.HARGA>data.HARGA.quantile(0.01))&(data.HARGA<data.HARGA.quantile(0.99))]

In [148]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [153]:
X = data.drop('HARGA' , axis=1)
y = data['HARGA']

In [154]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [158]:
data = data.fillna(0)

In [159]:
model = RandomForestRegressor()
model.fit(X_train,y_train)
hasil = model.predict(X_test)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values